In [2]:
import pandas as pd

df = pd.read_excel(r"D:\项目\重庆荣昌\区块链\平台在线交易流水.xlsx",sheet_name='Sheet1')
df= df.drop('地址交易记录', axis=1)
df = df.drop_duplicates()
df=df[df['代币符号']=='USDT']
# 初始化一个空的列表来存储结果行
result_rows = []

grouped_df = df.groupby(['发送方', '接收方'], as_index=False).agg({"金额": "sum"})


In [3]:
grouped_df

,发送方,接收方,金额
0,T9yVotmyQQMDmuQDRX1fbuhyMwbDxcxgNH,TAChDmZrSP4EFTmLfqKoakAx9TEA5Q8f3t,0
1,T9z8SvW2YS6BBvQdPexFhDB6Vf4JoDraA8,TAChDmZrSP4EFTmLfqKoakAx9TEA5Q8f3t,0.020101
2,T9zQMcwkQZB1RJb1cy4A29aCai7gnfrAA8,TAChDmZrSP4EFTmLfqKoakAx9TEA5Q8f3t,0.001004
3,T9zgCP1hemihqM1nMct3zWTaZHFgHp74pD,TFbJUS7J8tQ54JU7zPoTnrSjMgsnqrM39C,2500
4,T9zgCP1hemihqM1nMct3zWTaZHFgHp74pD,TNLbrz765R8VZVobVkdvYfq3jUrS4K9bX1,6008
...,...,...,...
8939,TZHFGhbzzwTiiu1EeDA9S4Qf3PXi66RgjG,TAChDmZrSP4EFTmLfqKoakAx9TEA5Q8f3t,0.0637
8940,TZHKBtA1eqs6moiUUSWRAwoNtKojzsRGjG,TFbJUS7J8tQ54JU7zPoTnrSjMgsnqrM39C,0.00143
8941,TZHTExP2KMLg8wHeNmLezHAUDwXGQxvoMM,TNLbrz765R8VZVobVkdvYfq3jUrS4K9bX1,300
8942,TZJ99g8mFhNsZX784j6N7BhrEYxaBJ7bQZ,TLGQZorgCVBqR43MRVNnkPTx9ubdmSUhes,741


In [4]:
for index, row in grouped_df.iterrows():
    pair_reversed = (row['接收方'], row['发送方'])
    if any((res_row['发送方'] == pair_reversed[0]) & (res_row['接收方'] == pair_reversed[1]) for res_row in result_rows):
        continue

    reversed_row = grouped_df.loc[(grouped_df['发送方'] == pair_reversed[0]) & (grouped_df['接收方'] == pair_reversed[1])]

    if not reversed_row.empty:
        net_amount = row['金额'] - reversed_row.iloc[0]['金额']
        if net_amount != 0:
            new_row = {'发送方': row['发送方'], '接收方': row['接收方'], '流向数额': net_amount}
            result_rows.append(new_row)
    else:
        new_row = {'发送方': row['发送方'], '接收方': row['接收方'], '流向数额': row['金额']}
        result_rows.append(new_row)

result = pd.DataFrame(result_rows)


In [17]:
result.groupby(['接收方'], as_index=False).agg({"流向数额": "count"})

,接收方,流向数额
0,T9z4tYMfRhKLagDXJEJ7pbCj9Mxo8MHiEf,1
1,T9z8SSW9dCWPmtvz5a2fXMEKtB2AoDraA8,1
2,T9zqhtHpyLSDpDJHZV8N2ikdzwKrEU1111,3
3,TA1w9TAPNdJzGXNzSfJp9oY73kFhPC1CWR,1
4,TA25RQSfRUJ4vAsdK9Br5r3xNpiyMUpz4G,1
...,...,...
2782,TZHTExP2KMLg8wHeNmLezHAUDwXGQxvoMM,1
2783,TZHpkBNXLGqJsFfWqXkerdFEjguU5RU8cG,1
2784,TZJ2nZgLbSigrzfDSBJx8qFpeesCDDsA8W,1
2785,TZJFBAyNEoNxSkaYNqmindsXU7xnz1aLy7,1


In [7]:
amount_5w=result[abs(result['流向数额'])>50000]
amount_5w

,发送方,接收方,流向数额
34,TAB1x9Np4Tzbt2jCg8gEMsDv1jnTvHAA1D,TAChDmZrSP4EFTmLfqKoakAx9TEA5Q8f3t,790007.56
36,TAB1x9Np4Tzbt2jCg8gEMsDv1jnTvHAA1D,TFbJUS7J8tQ54JU7zPoTnrSjMgsnqrM39C,1013817.90
37,TAB1x9Np4Tzbt2jCg8gEMsDv1jnTvHAA1D,TLGQZorgCVBqR43MRVNnkPTx9ubdmSUhes,627151.40
38,TAB1x9Np4Tzbt2jCg8gEMsDv1jnTvHAA1D,TNLbrz765R8VZVobVkdvYfq3jUrS4K9bX1,317666.89
39,TAB1x9Np4Tzbt2jCg8gEMsDv1jnTvHAA1D,TPaffDHPetyAMBr2wHEKqk1gigaaotdsah,585763.60
...,...,...,...
7867,TYqVQpYyUHWQLzpx5J5qbsS7458r6R8Fop,TLGQZorgCVBqR43MRVNnkPTx9ubdmSUhes,183125.00
7868,TYqVQpYyUHWQLzpx5J5qbsS7458r6R8Fop,TNLbrz765R8VZVobVkdvYfq3jUrS4K9bX1,107993.00
7870,TYqVQpYyUHWQLzpx5J5qbsS7458r6R8Fop,TVxfbZgroDs2oLVSoqPVd8xUfFx4WTGsJG,82162.00
7902,TZ247x5MqkEQ3ugQC38qHpDrN3CZKUuwHa,TVxfbZgroDs2oLVSoqPVd8xUfFx4WTGsJG,66500.00


In [12]:
amount_5w

,发送方,接收方,流向数额
34,TAB1x9Np4Tzbt2jCg8gEMsDv1jnTvHAA1D,TAChDmZrSP4EFTmLfqKoakAx9TEA5Q8f3t,790007.56
36,TAB1x9Np4Tzbt2jCg8gEMsDv1jnTvHAA1D,TFbJUS7J8tQ54JU7zPoTnrSjMgsnqrM39C,1013817.90
37,TAB1x9Np4Tzbt2jCg8gEMsDv1jnTvHAA1D,TLGQZorgCVBqR43MRVNnkPTx9ubdmSUhes,627151.40
38,TAB1x9Np4Tzbt2jCg8gEMsDv1jnTvHAA1D,TNLbrz765R8VZVobVkdvYfq3jUrS4K9bX1,317666.89
39,TAB1x9Np4Tzbt2jCg8gEMsDv1jnTvHAA1D,TPaffDHPetyAMBr2wHEKqk1gigaaotdsah,585763.60
...,...,...,...
7867,TYqVQpYyUHWQLzpx5J5qbsS7458r6R8Fop,TLGQZorgCVBqR43MRVNnkPTx9ubdmSUhes,183125.00
7868,TYqVQpYyUHWQLzpx5J5qbsS7458r6R8Fop,TNLbrz765R8VZVobVkdvYfq3jUrS4K9bX1,107993.00
7870,TYqVQpYyUHWQLzpx5J5qbsS7458r6R8Fop,TVxfbZgroDs2oLVSoqPVd8xUfFx4WTGsJG,82162.00
7902,TZ247x5MqkEQ3ugQC38qHpDrN3CZKUuwHa,TVxfbZgroDs2oLVSoqPVd8xUfFx4WTGsJG,66500.00


In [16]:
from graphviz import Digraph
import pandas as pd
import os 
# 以下这两行是手动进行环境变量配置，防止在本机的环境变量部署失败
#os.environ['PATH'] = os.pathsep + r"C:\ProgramData\Anaconda3\Scripts\graphviz\bin"
def get_color_send(amount):
    if amount >= 300000:
        return 'yellow'
    else:
        return 'grey'
def get_color_receive(amount_receive):
    if amount_receive >= 300000:
        return 'yellow'
    if amount_receive >= 100000:
        return 'green'
    else:
        return 'grey'
if __name__ == "__main__":    
    data_info=result.groupby(['接收方'], as_index=False).agg({"流向数额": "sum"})
    info_dict = data_info.set_index("接收方", drop=True).to_dict(orient="index")
    dot = Digraph('pictrue', format='pdf') # 创建photo图像 
    dot.attr(rankdir='LR')
    nodes = {} # 存储所有节点名称的字典    
    for index, row in amount_5w.iterrows():
        sender = row['发送方']
        receiver = row['接收方']
        amount = row['流向数额']
        
        if sender not in nodes.keys(): # 如果没有画过该节点 
            dot.node(sender, sender, style='filled', fillcolor='white')
            #photo.node(name=mother_name, fontname="Microsoft YaHei") # 画母公司节点        
            if receiver  not in nodes.keys(): 
                amount_receive=info_dict[receiver]['流向数额']
                #receiver_info=str(receiver)+'\n'+str(info_dict[receiver])
                dot.node(receiver, receiver, style='filled', fillcolor=get_color_receive(amount_receive))

               
                dot.edge(sender, receiver,label=f"{amount}")
              
    dot.render(filename='流向5万', directory=r'D:\项目\重庆荣昌\222')# 打开图片

In [14]:
data_info

,接收方,流向数额
0,T9z4tYMfRhKLagDXJEJ7pbCj9Mxo8MHiEf,1364.000000
1,T9z8SSW9dCWPmtvz5a2fXMEKtB2AoDraA8,1005.982100
2,T9zqhtHpyLSDpDJHZV8N2ikdzwKrEU1111,1661.298060
3,TA1w9TAPNdJzGXNzSfJp9oY73kFhPC1CWR,0.000000
4,TA25RQSfRUJ4vAsdK9Br5r3xNpiyMUpz4G,92.253521
...,...,...
2782,TZHTExP2KMLg8wHeNmLezHAUDwXGQxvoMM,28.639240
2783,TZHpkBNXLGqJsFfWqXkerdFEjguU5RU8cG,104.654895
2784,TZJ2nZgLbSigrzfDSBJx8qFpeesCDDsA8W,4640.900000
2785,TZJFBAyNEoNxSkaYNqmindsXU7xnz1aLy7,251.532846


In [28]:
#result.groupby(['接收方'], as_index=False)['流向数额'].agg([('cnt','count'),('hj','sum')])
receive_times=result.groupby(['接收方'], as_index=False).agg({'流向数额':[('cnt','count'),('hj','sum')]})

接收方 流向数额             
                                          cnt           hj
0     T9z4tYMfRhKLagDXJEJ7pbCj9Mxo8MHiEf    1  1364.000000
1     T9z8SSW9dCWPmtvz5a2fXMEKtB2AoDraA8    1  1005.982100
2     T9zqhtHpyLSDpDJHZV8N2ikdzwKrEU1111    3  1661.298060
3     TA1w9TAPNdJzGXNzSfJp9oY73kFhPC1CWR    1     0.000000
4     TA25RQSfRUJ4vAsdK9Br5r3xNpiyMUpz4G    1    92.253521
...                                  ...  ...          ...
2782  TZHTExP2KMLg8wHeNmLezHAUDwXGQxvoMM    1    28.639240
2783  TZHpkBNXLGqJsFfWqXkerdFEjguU5RU8cG    1   104.654895
2784  TZJ2nZgLbSigrzfDSBJx8qFpeesCDDsA8W    1  4640.900000
2785  TZJFBAyNEoNxSkaYNqmindsXU7xnz1aLy7    1   251.532846
2786  TZJGs5ZFkjgemT1uc6Vge9BRHuvPWDsA8W    1  1742.900000

[2787 rows x 3 columns]

In [48]:
receive_times = result.groupby(['接收方'], as_index=False).agg({'流向数额': [('cnt', 'count'), ('hj', 'sum')]})
filtered_receive_times = receive_times[~
    ((receive_times[('流向数额', 'hj')] <= 50000) |
    ((receive_times[('流向数额', 'cnt')] == 1) & (receive_times[('流向数额', 'hj')] <= 100000)))
]
print(filtered_receive_times)
need_receive=filtered_receive_times['接收方']

                                     接收方  流向数额              
                                           cnt            hj
11    TA6DhFgeTRzbyiMNtYpU4JHaHiRH4RGCxj     6  2.904026e+05
29    TAChDmZrSP4EFTmLfqKoakAx9TEA5Q8f3t  1489  3.237947e+07
46    TALJVAnsNR3o2uNvGv586txQUQPqH7jy7i     7  3.741042e+05
98    TAumGQFegqsp3BtZ44buaqoDSHjceXN3bn     5  1.499420e+05
141   TBKHsnxdpNM55aW4cHeis6JskVoqnaPRHA     2  2.500000e+05
...                                  ...   ...           ...
2563  TXbNd8JnnWrfUbmWAxWY1X9WcPYWKsXoGj     2  5.385295e+04
2632  TY8mHADJEPehSiT9CHrn16Rqus68hD61ie     2  7.738506e+04
2651  TYGZ4rfidvHpSpKrY3dyvoK8MgJZ5YxBmb     4  7.552580e+05
2718  TYstyiNGLGXLxf1QJcCh6YdZ77q1GAfTmm     1  1.242485e+05
2774  TZDEwXCPtd4v5Kpht5UX7Fxjbd4rxLsK5F     6  2.718082e+05

[95 rows x 3 columns]


In [49]:
need_receive

11      TA6DhFgeTRzbyiMNtYpU4JHaHiRH4RGCxj
29      TAChDmZrSP4EFTmLfqKoakAx9TEA5Q8f3t
46      TALJVAnsNR3o2uNvGv586txQUQPqH7jy7i
98      TAumGQFegqsp3BtZ44buaqoDSHjceXN3bn
141     TBKHsnxdpNM55aW4cHeis6JskVoqnaPRHA
                       ...                
2563    TXbNd8JnnWrfUbmWAxWY1X9WcPYWKsXoGj
2632    TY8mHADJEPehSiT9CHrn16Rqus68hD61ie
2651    TYGZ4rfidvHpSpKrY3dyvoK8MgJZ5YxBmb
2718    TYstyiNGLGXLxf1QJcCh6YdZ77q1GAfTmm
2774    TZDEwXCPtd4v5Kpht5UX7Fxjbd4rxLsK5F
Name: 接收方, Length: 95, dtype: object

In [51]:
amount_5w2_10w1=amount_5w.merge(need_receive,on='接收方',how='inner')

In [52]:
from graphviz import Digraph
import pandas as pd
import os 
# 以下这两行是手动进行环境变量配置，防止在本机的环境变量部署失败
#os.environ['PATH'] = os.pathsep + r"C:\ProgramData\Anaconda3\Scripts\graphviz\bin"
def get_color_send(amount):
    if amount >= 300000:
        return 'yellow'
    else:
        return 'grey'
def get_color_receive(amount_receive):
    if amount_receive >= 300000:
        return 'yellow'
    if amount_receive >= 100000:
        return 'green'
    else:
        return 'grey'
if __name__ == "__main__":    
    data_info=result.groupby(['接收方'], as_index=False).agg({"流向数额": "sum"})
    info_dict = data_info.set_index("接收方", drop=True).to_dict(orient="index")
    dot = Digraph('pictrue', format='pdf') # 创建photo图像 
    dot.attr(rankdir='LR')
    nodes = {} # 存储所有节点名称的字典    
    for index, row in amount_5w2_10w1.iterrows():
        sender = row['发送方']
        receiver = row['接收方']
        amount = row['流向数额']
        
        if sender not in nodes.keys(): # 如果没有画过该节点 
            dot.node(sender, sender, style='filled', fillcolor='white')
            #photo.node(name=mother_name, fontname="Microsoft YaHei") # 画母公司节点        
            if receiver  not in nodes.keys(): 
                amount_receive=info_dict[receiver]['流向数额']
                #receiver_info=str(receiver)+'\n'+str(info_dict[receiver])
                dot.node(receiver, receiver, style='filled', fillcolor=get_color_receive(amount_receive))

               
                dot.edge(sender, receiver,label=f"{amount}")
              
    dot.render(filename='amount_5w2_10w1', directory=r'D:\项目\重庆荣昌\222')# 打开图片